<a href="https://colab.research.google.com/github/aksht7/Classifier-On-IMDB_Dataset/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)
#tf.enable_eager_execution()
imdb,info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data,test_data=imdb['train'],imdb['test']
training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type="post"
oov_tok="<OOV>"

tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode_review(padded[0]))
print(training_sentences[0])

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


2.2.0
? ? b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The m

In [23]:
num_epochs=10
model.fit(padded,training_labels_final,epochs=num_epochs, validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 1.0256e-04 - accuracy: 1.0000 - val_loss: 0.8432 - val_accuracy: 0.8258
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 6.3704e-05 - accuracy: 1.0000 - val_loss: 0.8806 - val_accuracy: 0.8256
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 3.9880e-05 - accuracy: 1.0000 - val_loss: 0.9138 - val_accuracy: 0.8258
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 2.4944e-05 - accuracy: 1.0000 - val_loss: 0.9523 - val_accuracy: 0.8254
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 1.5791e-05 - accuracy: 1.0000 - val_loss: 0.9852 - val_accuracy: 0.8252
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 9.9622e-06 - accuracy: 1.0000 - val_loss: 1.0187 - val_accuracy: 0.8252
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 6.3302e-06 - accuracy: 1.0000 - val_loss: 1.0541 -

In [21]:
e=model.layers[0]
weights=e.get_weights()[0]
print('shape ',weights.shape)

shape  (10000, 16)


In [0]:
import io
out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,1000):
    word=reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')